<a href="https://colab.research.google.com/github/ejyepezm/PPIA/blob/main/unidad_3_aplicaciones_python/1_Pipelines_ScikitLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💊 Cápsula 1: El Modelo Robusto (ML Pipelines)
**Tema:** Scikit-learn Pipelines, ColumnTransformer y Prevención de Data Leakage.

## 1. El enemigo silencioso: Data Leakage

Cuando entrenas un modelo, sigues pasos: Imputar nulos -> Escalar datos -> Entrenar.
El error novato #1 es hacer esto:
1.  Escalar TODO el dataset.
2.  Dividir en Train/Test.

**¿El problema?** Al escalar todo junto, el conjunto de Test (el futuro) "contaminó" el promedio y la desviación estándar del conjunto de Train. Tu modelo sabe cosas del futuro que no debería saber.

### La Solución: Scikit-learn Pipeline
Un `Pipeline` encadena estos pasos en un solo objeto.
*   Cuando llamas `fit()`: Aprende los promedios SOLO de los datos de entrenamiento.
*   Cuando llamas `predict()`: Aplica esas transformaciones guardadas a los nuevos datos automáticamente.

**ColumnTransformer:** Nos permite aplicar transformaciones diferentes a columnas numéricas (Escalar) y categóricas (OneHotEncoding) en el mismo flujo.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# --- DATOS DE JUGUETE ---
# Imaginemos datos médicos: Edad (numérico) y Resultado (Target)
data = pd.DataFrame({
    'Edad': [25, 30, 45, 35, 60, None, 22, 50], # Tiene un nulo
    'Colesterol': [180, 200, 240, 210, 290, 190, 170, 250],
    'Enfermedad': [0, 0, 1, 0, 1, 0, 0, 1]
})

# Dividimos ANTES de tocar nada (Regla de oro)
X = data[['Edad', 'Colesterol']]
y = data['Enfermedad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("--- CONSTRUYENDO PIPELINE ---")

# Creamos el tubo de procesamiento
# Paso 1: Rellenar nulos con la media
# Paso 2: Escalar (StandardScaler)
# Paso 3: Modelo (Regresión Logística)
pipeline_medico = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

# Entrenamos TODO el flujo de una vez
pipeline_medico.fit(X_train, y_train)
print("✅ Pipeline entrenado exitosamente.")

# Predecimos (El pipeline se encarga de imputar y escalar el test set automáticamente)
predicciones = pipeline_medico.predict(X_test)
print(f"Predicciones: {predicciones}")
print(f"Accuracy: {accuracy_score(y_test, predicciones):.2f}")

## 🔥 Micro-Desafío: El "ColumnTransformer"

En la vida real, los datos son mixtos. Tienes columnas numéricas (Edad) y categóricas (Ciudad). No puedes escalar una ciudad ("Madrid"), y no puedes hacer OneHotEncoding de una edad.

**Tu Misión:**
Tienes un dataset de Clientes con:
*   Numéricas: `Gasto`, `Visitas`
*   Categóricas: `Genero`

Construye un **Preprocesador Híbrido** usando `ColumnTransformer`:
1.  A las numéricas: Aplica `SimpleImputer` (media) y `StandardScaler`.
2.  A las categóricas: Aplica `SimpleImputer` (moda/most_frequent) y `OneHotEncoder`.
3.  Une todo en un Pipeline final con un modelo `RandomForestClassifier`.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Datos mixtos
df_reto = pd.DataFrame({
    'Gasto': [100, 200, None, 300, 150],
    'Visitas': [1, 5, 2, 8, 3],
    'Genero': ['F', 'M', 'F', None, 'M'],
    'Compra': [0, 1, 0, 1, 0]
})

X = df_reto.drop('Compra', axis=1)
y = df_reto['Compra']

# Definimos qué columnas son cuáles
cols_num = ['Gasto', 'Visitas']
cols_cat = ['Genero']

# --- TU CÓDIGO AQUÍ ---

# 1. Pipeline para Numéricas
pipe_num = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 2. Pipeline para Categóricas
pipe_cat = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')) # handle_unknown es clave en producción
])

# 3. El Transformador de Columnas (El cerebro que divide)
# TODO: Completa el ColumnTransformer
# preprocesador = ColumnTransformer([
#     ('num', pipe_num, cols_num),
#     ('cat', ..., ...)
# ])
preprocesador = None # REEMPLAZA ESTO

# 4. Pipeline Final (Preprocesador + Modelo)
# TODO: Crea el Pipeline final usando RandomForestClassifier
pipeline_final = None # REEMPLAZA ESTO

# --- VALIDACIÓN (No tocar) ---
try:
    if pipeline_final:
        pipeline_final.fit(X, y)
        print("✅ ¡Éxito! El pipeline híbrido entrenó correctamente.")
        print("Clases del modelo:", pipeline_final.named_steps['model'].classes_)
    else:
        print("❌ Falta definir el pipeline_final.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Pista: Revisa los nombres de las columnas y los pasos del ColumnTransformer.")